###Uploading files

In [40]:
from google.colab import files
small_data = files.upload()
items_txt = files.upload()

Saving GroceryStore.csv to GroceryStore (1).csv


Saving Items.txt to Items (1).txt


##Reading datasets

In [41]:
import pandas as pd
import io
small_dataset = pd.read_csv(io.BytesIO(small_data['GroceryStore.csv']),header=None)
unique_items = pd.read_csv(io.BytesIO(items_txt['Items.txt']),header = None)
transactions = []
for i in range(len(small_dataset[0])):
  transactions.append(small_dataset[0][i].split(','))
unique_items_list = []
for i in range(12):
  unique_items_list.append(unique_items[i][0].strip())
transactions,unique_items_list #printing transactions and unique items in simple way

([['MILK', 'BREAD', 'BISCUIT'],
  ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
  ['BREAD', 'TEA', 'BOURNVITA'],
  ['JAM', 'MAGGI', 'BREAD', 'MILK'],
  ['MAGGI', 'TEA', 'BISCUIT'],
  ['BREAD', 'TEA', 'BOURNVITA'],
  ['MAGGI', 'TEA', 'CORNFLAKES'],
  ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
  ['JAM', 'MAGGI', 'BREAD', 'TEA'],
  ['BREAD', 'MILK'],
  ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
  ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
  ['COFFEE', 'SUGAR', 'BOURNVITA'],
  ['BREAD', 'COFFEE', 'COCK'],
  ['BREAD', 'SUGAR', 'BISCUIT'],
  ['COFFEE', 'SUGAR', 'CORNFLAKES'],
  ['BREAD', 'SUGAR', 'BOURNVITA'],
  ['BREAD', 'COFFEE', 'SUGAR'],
  ['BREAD', 'COFFEE', 'SUGAR'],
  ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']],
 ['JAM',
  'MAGGI',
  'SUGAR',
  'COFFEE',
  'CHEESE',
  'TEA',
  'BOURNVITA',
  'CORNFLAKES',
  'BREAD',
  'BISCUIT',
  'MILK',
  'COCK'])

##Frequent itemsets and Apriori

In [48]:
import itertools
from itertools import combinations, chain
from collections import defaultdict

numberOftransactions = len(transactions)
#Finding number of occurence
def occured_number(itemset, transactions):
  k = 0
  for i in range(numberOftransactions):
    if set(itemset).issubset(set(transactions[i])):
      k += 1
  return k

#Finding itemsets which occured at least (min support) times
def frequent_items(itemset,transactions, min_supp, pr_rejected):
  numberOftransactions = len(transactions)
  fr_itemsets = []
  sup_count = []
  new_rejected = []
  key = len(pr_rejected.keys())
  
  for i in range(len(itemset)):
    rej_before = False
    if key > 0:
      for j in pr_rejected[key]:
        if set(j).issubset(set(itemset[i])):
          rej_before = True
          break
    if not rej_before:
      count = occured_number(itemset[i],transactions)
      if count / numberOftransactions >= min_supp:
        fr_itemsets.append(itemset[i])
        sup_count.append(count)
      else:
        new_rejected.append(itemset[i])
  return fr_itemsets, sup_count, new_rejected

 
sizeOfset = 1
itemsets = {}
itemsets.update({sizeOfset:[[i] for i in unique_items_list]})

#initialization of frequent itemsets
fr_itemsets = {}
sup_count = {}
rejected = {sizeOfset:[]}
min_supp = 2 / 20
min_conf = 0.6
frequent_itemsets,supp,new_rejected = frequent_items(itemsets[sizeOfset],transactions,min_supp,rejected)
rejected.update({sizeOfset : new_rejected})
fr_itemsets.update({sizeOfset : frequent_itemsets})
sup_count.update({sizeOfset : supp})


fr_itemsets[1],sup_count[1]



([['JAM'],
  ['MAGGI'],
  ['SUGAR'],
  ['COFFEE'],
  ['TEA'],
  ['BOURNVITA'],
  ['CORNFLAKES'],
  ['BREAD'],
  ['BISCUIT'],
  ['MILK'],
  ['COCK']],
 [2, 5, 6, 8, 7, 4, 6, 13, 7, 5, 3])

In [ ]:
##Next step is grouping by bigger size starting with two (K-ITEMSETS)

In [50]:
#Functions for candidate joining
def two_joint_itemset(set_one, set_two, unique_items_list):
  set_one.sort(key = lambda x: unique_items_list.index(x))
  set_two.sort(key = lambda x: unique_items_list.index(x))
  for i in range(len(set_one)-1):
    if set_one[i] != set_two[i]:
      return []
  if (unique_items_list.index(set_one[-1]) < unique_items_list.index(set_two[-1])):
    return set_one + [set_two[-1]]
  return []


def joint_itemset(setOfitemsets,unique_items_list):
  itemsets = []
  for i in range(len(setOfitemsets)):
    for j in range(i+1, len(setOfitemsets)):
      outOfit = two_joint_itemset(setOfitemsets[i], setOfitemsets[j],unique_items_list)
      if len(outOfit) > 0:
        itemsets.append(outOfit)
  return itemsets
k_itemset = sizeOfset + 1
key_link = 0 
#printing candidates and frequent items and their occurence
while key_link == 0:
  itemsets.update({k_itemset : joint_itemset(fr_itemsets[k_itemset-1],unique_items_list)})
  print("Table itemsets{}: \n".format(k_itemset))
  print(itemsets[k_itemset],[occured_number(itemset,transactions) for itemset in itemsets[k_itemset]],"\n")
  frequent_itemsets,supp,new_rejected=frequent_items(itemsets[k_itemset],transactions,min_supp,rejected)
  fr_itemsets.update({k_itemset : frequent_itemsets})
  rejected.update({k_itemset : new_rejected})
  sup_count.update({k_itemset : supp})
  if len(fr_itemsets[k_itemset]) == 0:
    key_link = 1
  else:
    print("Table itemsets{}: \n".format(k_itemset))
    print(fr_itemsets[k_itemset],sup_count[k_itemset],"\n")
  k_itemset += 1


Table itemsets2: 

[['JAM', 'MAGGI'], ['JAM', 'SUGAR'], ['JAM', 'COFFEE'], ['JAM', 'TEA'], ['JAM', 'BOURNVITA'], ['JAM', 'CORNFLAKES'], ['JAM', 'BREAD'], ['JAM', 'BISCUIT'], ['JAM', 'MILK'], ['JAM', 'COCK'], ['MAGGI', 'SUGAR'], ['MAGGI', 'COFFEE'], ['MAGGI', 'TEA'], ['MAGGI', 'BOURNVITA'], ['MAGGI', 'CORNFLAKES'], ['MAGGI', 'BREAD'], ['MAGGI', 'BISCUIT'], ['MAGGI', 'MILK'], ['MAGGI', 'COCK'], ['SUGAR', 'COFFEE'], ['SUGAR', 'TEA'], ['SUGAR', 'BOURNVITA'], ['SUGAR', 'CORNFLAKES'], ['SUGAR', 'BREAD'], ['SUGAR', 'BISCUIT'], ['SUGAR', 'MILK'], ['SUGAR', 'COCK'], ['COFFEE', 'TEA'], ['COFFEE', 'BOURNVITA'], ['COFFEE', 'CORNFLAKES'], ['COFFEE', 'BREAD'], ['COFFEE', 'BISCUIT'], ['COFFEE', 'MILK'], ['COFFEE', 'COCK'], ['TEA', 'BOURNVITA'], ['TEA', 'CORNFLAKES'], ['TEA', 'BREAD'], ['TEA', 'BISCUIT'], ['TEA', 'MILK'], ['TEA', 'COCK'], ['BOURNVITA', 'CORNFLAKES'], ['BOURNVITA', 'BREAD'], ['BOURNVITA', 'BISCUIT'], ['BOURNVITA', 'MILK'], ['BOURNVITA', 'COCK'], ['CORNFLAKES', 'BREAD'], ['CORNFLAKES', 

##Association rules

In [52]:
#Getting all possible combinations
def set_power(subset):
  return list(chain.from_iterable(combinations(subset,r) for r in range(1,len(subset)+1)))

#Function for generating a rule
def rule_gen(X,X_Y,Y,confidence,supp,lift,numberOftransactions):
  rule = ""
  rule += "Fr. Itemset: {}\n".format(X)
  rule += "    Rule: {} -> {} \n".format(list(Y),list(X_Y))
  rule += "    Confidence: {0:2.3f} \n".format(confidence)
  rule += "    Support: {0:2.3f} \n".format(supp / numberOftransactions)
  rule += "    Lift: {0:2.3f} \n".format(lift)
  return rule

rules = ""
for i in range(1, len(fr_itemsets)):
  for j in range(len(fr_itemsets[i])):
    subset = list(set_power(set(fr_itemsets[i][j])))
    subset.pop()
    for k in subset:
      Y = set(k)
      X = set(fr_itemsets[i][j])
      X_Y = set(X-Y)
      x_supp = occured_number(X,transactions)
      x_y_supp =  occured_number(X_Y,transactions)
      confidence = x_supp / occured_number(Y,transactions)
      lift = confidence / (x_y_supp / numberOftransactions)
      if confidence >= min_conf and x_supp >= min_supp:
        rules += rule_gen(X,X_Y,Y,confidence,x_supp,lift,numberOftransactions)

print(rules)

Fr. Itemset: {'MAGGI', 'JAM'}
    Rule: ['JAM'] -> ['MAGGI'] 
    Confidence: 1.000 
    Support: 0.100 
    Lift: 4.000 
Fr. Itemset: {'JAM', 'BREAD'}
    Rule: ['JAM'] -> ['BREAD'] 
    Confidence: 1.000 
    Support: 0.100 
    Lift: 1.538 
Fr. Itemset: {'TEA', 'MAGGI'}
    Rule: ['MAGGI'] -> ['TEA'] 
    Confidence: 0.800 
    Support: 0.200 
    Lift: 2.286 
Fr. Itemset: {'MAGGI', 'BREAD'}
    Rule: ['MAGGI'] -> ['BREAD'] 
    Confidence: 0.600 
    Support: 0.150 
    Lift: 0.923 
Fr. Itemset: {'COFFEE', 'SUGAR'}
    Rule: ['SUGAR'] -> ['COFFEE'] 
    Confidence: 0.667 
    Support: 0.200 
    Lift: 1.667 
Fr. Itemset: {'SUGAR', 'BREAD'}
    Rule: ['SUGAR'] -> ['BREAD'] 
    Confidence: 0.667 
    Support: 0.200 
    Lift: 1.026 
Fr. Itemset: {'CORNFLAKES', 'COFFEE'}
    Rule: ['CORNFLAKES'] -> ['COFFEE'] 
    Confidence: 0.667 
    Support: 0.200 
    Lift: 1.667 
Fr. Itemset: {'COCK', 'COFFEE'}
    Rule: ['COCK'] -> ['COFFEE'] 
    Confidence: 1.000 
    Support: 0.150 
    Lif